In [2]:
import pandas as pd
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']
dtype = {}
for df in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=10):
    for var in df.columns:
        if var not in numericos:
            dtype[var] = 'int8'
    break

In [3]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12


In [13]:
from	keras import	models,	layers

model=models.Sequential()
model.add(layers.Dense(16,	activation='relu',	input_shape=(auxPred.shape[1]-1,)))
model.add(layers.Dense(16,	activation='relu'))
model.add(layers.Dense(1,	activation='sigmoid'))

In [14]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

In [16]:
trainX = auxPred.loc[:499999, auxPred.columns != 'HasDetections']
valX = auxPred.loc[500000:549999, auxPred.columns != 'HasDetections']
testX = auxPred.loc[550000:599999, auxPred.columns != 'HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']
testY = auxPred.loc[550000:599999,'HasDetections']

In [17]:
history =	model.fit(trainX,
trainY,
epochs=5,
batch_size=512,
validation_data=(valX,	valY))

Instructions for updating:
Use tf.cast instead.
Train on 500000 samples, validate on 50000 samples
Epoch 1/5
500000/500000 [==============================] - 23s 46us/step - loss: 7.9672 - acc: 0.5003 - val_loss: 7.9766 - val_acc: 0.4997
Epoch 2/5
500000/500000 [==============================] - 19s 38us/step - loss: 7.9672 - acc: 0.5002 - val_loss: 7.9766 - val_acc: 0.4997
Epoch 3/5
500000/500000 [==============================] - 19s 38us/step - loss: 7.9672 - acc: 0.5002 - val_loss: 7.9766 - val_acc: 0.4997
Epoch 4/5
500000/500000 [==============================] - 19s 38us/step - loss: 7.9672 - acc: 0.5002 - val_loss: 7.9766 - val_acc: 0.4997
Epoch 5/5
500000/500000 [==============================] - 19s 37us/step - loss: 7.9672 - acc: 0.5002 - val_loss: 7.9766 - val_acc: 0.4997


In [ ]:
from sklearn.metrics import roc_curve, auc
pred = model.predict(testX)
false_positive_rate, true_positive_rate, thresholds = roc_curve(testY, pred)
score = auc(false_positive_rate, true_positive_rate)